### Image Classification: Crack Detection 📷🏫

Using a Convolutional Neural Network to detect cracks in concrete.

In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import os
import glob

import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchmetrics import Accuracy, MeanSquaredError, R2Score

pd.set_option('display.max.columns', 500)

Let's start by reading in the images into memory. Using the ImageFolder from `torchvision`, the images must be sorted like following:

data/
    Negative/
        image1.jpg
        image2.jpg
        ...
    Positive/
        image1.jpg
        image2.jpg
        ...        

In [2]:
# Define the root directory where your images are located
root = "C:/Users/mathi/Downloads/CNN"

# Create a PyTorch dataset from the ImageFolder class
dataset = ImageFolder(root, transform=None)

Creating the dataloaders from a random split:

In [3]:
# Define the sizes of the training, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Use random_split to split the dataset into non-overlapping training, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create dataloaders for each set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# Create a PyTorch dataloader to load the images in batches
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
class CNNClassifier(pl.LightningModule):
    
    def __init__(self):
        super(CNNClassifier, self).__init__()

        # Define the convolutional and pooling layers
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define the fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        # Apply convolutional and pooling layers
        x = self.layers(x)

        # Flatten the output from convolutional layers
        x = x.view(-1, 128 * 4 * 4)

        # Apply fully connected layers
        x = self.fc_layers(x)
        
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss)
        self.log('val_acc', acc)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        return loss
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            y_hat = self(x)
            y_hat = y_hat.argmax(dim=1)
        return y_hat
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [6]:
model = CNNClassifier()

# Instantiate a PyTorch Lightning Trainer object
trainer = pl.Trainer(max_epochs=10, gpus=0)

# Train the model
trainer.fit(model, train_loader, val_loader)

# Test the model on the test set
result = trainer.test(test_dataloaders=test_loader)

# Print the test set accuracy
test_acc = result[0]['test_acc']
print(f"Test accuracy: {test_acc:.4f}")

C:\Users\mathi\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type       | Params
-----------------------------------------
0 | layers    | Sequential | 93.2 K
1 | fc_layers | Sequential | 1.1 M 
-----------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.573     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\mathi\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>